# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [16]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [125]:
def getNGrams(words,n):
    ngrams = {}
    for word in words:
        # word="$"*(n-1)+word+"$"*(n-1)
        if len(word) <n:
            continue
        for i in range(len(word)-n+1):
            ngram = word[i:i+n]
            if ngram not in ngrams:
                ngrams[ngram] = 1
            else:
                ngrams[ngram]+=1
    return ngrams


In [126]:
word=["abc"]
# guessed=list(set(word)-set("_"))
print(getNGrams(word,1))

{'a': 1, 'b': 1, 'c': 1}


In [161]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        # full_dictionary_location = "words_250000_train.txt"
        # self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        # self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        dictionary_file_location="words_250000_train.txt"
        text_file = open(dictionary_file_location,"r")
        words = text_file.read().splitlines()
        text_file.close()
        self.ngrams=[getNGrams(words,i) for i in range(1,6)]

        # self.current_dictionary = []
    def buildDict(self):
        dictionary_file_location="words_250000_train.txt"
        text_file = open(dictionary_file_location,"r")
        words = text_file.read().splitlines()
        text_file.close()
        
        self.ngrams=[getNGrams(words,i) for i in range(1,5)]
        
    def ngramprob(self,word,letter ):
        # print("________________________: word:",word,"letter:",letter)
        denominator=0
        underscroll_location=word.find("_")
        # if len(word)>2:
        #     print("word:",word,"letter:",letter)
        #     for l in string.ascii_lowercase:
        #         print("l:",l)
        #         print(self.ngrams[len(word)-1][word[:underscroll_location]+l+word[underscroll_location+1:]])

        for l in string.ascii_lowercase:
            try:
                denominator+=self.ngrams[len(word)-1][word[:underscroll_location]+l+word[underscroll_location+1:]]
            except:
                pass
        # print("____________:",word)
        # if len(word)>3:
        #     print("word with length >3:",word)
        # if word=="a_s":
        #     print("letter:",letter)
        #     print("denominator:",denominator)
        #     print("numerator:",self.ngrams[len(word)-1][word[:underscroll_location]+letter+word[underscroll_location+1:]])
        return self.ngrams[len(word)-1][word[:underscroll_location]+letter+word[underscroll_location+1:]]/denominator

        
    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # clean_word = word[::2].replace("_",".")
        # guessed=list(set(word)-set("_"))
        word=word[::2]
        available = list(set(string.ascii_lowercase) - set(self.guessed_letters))
        probs=[]
        contributions=[0,0,0,0]
        for char in available:
            char_prob=0
            for i in range(len(word)):
                if word[i]!="_":
                    continue
                # find the longest continious string without any "_"
                # 4gram
                try:
                    if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_":
                        contributions[3]+=self.ngramprob(word[i-3:i+1],char)
                        char_prob+=self.ngramprob(word[i-3:i+1],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_":
                        contributions[3]+=self.ngramprob(word[i-2:i+2],char)
                        char_prob+=self.ngramprob(word[i-2:i+2],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_":
                        contributions[3]+=self.ngramprob(word[i-1:i+3],char)
                        char_prob+=self.ngramprob(word[i-1:i+3],char)
                except:
                    pass
                try: 
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_":
                        contributions[3]+=self.ngramprob(word[i:i+4],char)
                        char_prob+=self.ngramprob(word[i:i+4],char) 
                except:
                    pass
                # 3gram
                try:
                    if word[i-1]!="_" and word[i-2]!="_":
                        contributions[2]+=self.ngramprob(word[i-2:i+1],char)
                        char_prob+=self.ngramprob(word[i-2:i+1],char)
                except:
                    pass
                try:
                    # a_b
                    if word[i+1]!="_" and word[i-1]!="_":
                        contributions[2]+=self.ngramprob(word[i-1:i+2],char)
                        char_prob+=self.ngramprob(word[i-1:i+2],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_":
                        contributions[2]+=self.ngramprob(word[i:i+3],char)
                        char_prob+=self.ngramprob(word[i:i+3],char)
                except:
                    pass
                    # 2gram
                try:
                    if word[i-1]!="_":
                        contributions[1]+=self.ngramprob(word[i-1:i+1],char)
                        char_prob+=self.ngramprob(word[i-1:i+1],char)
                except:
                    pass
                try:
                    if word[i+1]!="_":
                        contributions[1]+=self.ngramprob(word[i:i+2],char)
                        char_prob+=self.ngramprob(word[i:i+2],char)
                except:
                    pass
                # 1gram
                char_prob+=self.ngramprob(word[i:i+1],char)
                contributions[0]+=self.ngramprob(word[i:i+1],char)
            probs.append(char_prob)
        # create sorted dictionary of probabilities
        probs_dict=dict(zip(available,probs))
        probs_sorted=sorted(probs_dict.items(), key=lambda x: x[1], reverse=True)
        # iterate on probs_sorted
        for i in probs_sorted:
            # round the probability to 3 decimal places
            # i[1]=round()
            print(i[0],round(i[1],2),end=", ")
        # for i in range(len(probs)):
        #     # print upto 2 decimal places
        #     # probs[i]=round(probs[i],2)
        #     print(probs_sorted[i],round(probs_sorted[i],2),end=" ")
        print()
        print("contributions:",contributions)
        return available[probs.index(max(probs))]
            
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        # self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    def offline(self,wordtobeguessed):
        self.guessed_letters=[]
        tries_remains=6
        word = "_ " * len(wordtobeguessed)
        assert len(word) == len(wordtobeguessed) *2
        while tries_remains>0:
                # get guessed letter from user code
            guess_letter = self.guess(word)
            flag=False
            for index in range(len(wordtobeguessed)):
                if wordtobeguessed[index]==guess_letter:
                    word=word[:2*index]+guess_letter+word[2*index+1:]
                    flag=True
            if flag==False:
                tries_remains-=1
                print("Wrong guess, you have",tries_remains,"tries left")
            if "_" not in word:
                print("You won!")
                break
            # append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            # time.sleep(0.5)
            print("Guessed letter: {0}".format(guess_letter))
            print("Current word: {0}".format(word))
            
            # tries_remains -=1
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [162]:
api = HangmanAPI(access_token="1351d29da90b3d587ad0f8b1720f69", timeout=2000)


In [159]:
api.offline("hangman")

e 0.77, i 0.61, a 0.59, n 0.5, o 0.49, r 0.49, s 0.49, t 0.45, l 0.4, c 0.29, u 0.25, d 0.25, p 0.22, m 0.2, h 0.19, g 0.17, y 0.14, b 0.13, f 0.09, v 0.07, k 0.06, w 0.06, z 0.03, x 0.02, q 0.01, j 0.01, 
contributions: [7.000000000000002, 0, 0, 0]
Wrong guess, you have 5 tries left
Guessed letter: e
Current word: _ _ _ _ _ _ _ 
i 0.61, a 0.59, n 0.5, o 0.49, r 0.49, s 0.49, t 0.45, l 0.4, c 0.29, u 0.25, d 0.25, p 0.22, m 0.2, h 0.19, g 0.17, y 0.14, b 0.13, f 0.09, v 0.07, k 0.06, w 0.06, z 0.03, x 0.02, q 0.01, j 0.01, 
contributions: [6.229924423907606, 0, 0, 0]
Wrong guess, you have 4 tries left
Guessed letter: i
Current word: _ _ _ _ _ _ _ 
a 0.59, n 0.5, o 0.49, r 0.49, s 0.49, t 0.45, l 0.4, c 0.29, u 0.25, d 0.25, p 0.22, m 0.2, h 0.19, g 0.17, y 0.14, b 0.13, f 0.09, v 0.07, k 0.06, w 0.06, z 0.03, x 0.02, q 0.01, j 0.01, 
contributions: [5.621276613769361, 0, 0, 0]
Guessed letter: a
Current word: _ a _ _ _ a _ 
r 0.88, n 0.83, l 0.8, t 0.8, s 0.56, c 0.52, m 0.4, o 0.39, p 

In [112]:
# words=["ab","bc"]

# print(ngrams)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [163]:
[total_practice_runs_,total_recorded_runs,total_recorded_successes,total_practice_successes_] = api.my_status()
for _ in range(100):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
print(total_practice_successes-total_practice_successes_)

Successfully start a new game! Game ID: f33e7647b251. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
e 1.43, i 1.13, a 1.1, n 0.93, o 0.92, r 0.91, s 0.91, t 0.84, l 0.75, c 0.55, u 0.47, d 0.46, p 0.4, m 0.38, h 0.36, g 0.32, y 0.25, b 0.24, f 0.16, v 0.13, k 0.11, w 0.11, z 0.05, x 0.04, q 0.02, j 0.02, 
contributions: [13.000000000000009, 0, 0, 0]
Guessing letter: e
Sever response: {'game_id': 'f33e7647b251', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ e _ _ _ _ _ '}
r 1.15, i 1.09, a 1.08, n 1.06, s 1.04, t 0.95, o 0.87, l 0.86, d 0.6, c 0.58, u 0.46, p 0.44, m 0.44, h 0.38, g 0.34, b 0.26, y 0.25, v 0.18, f 0.18, k 0.13, w 0.12, z 0.07, x 0.05, j 0.03, q 0.03, 
contributions: [10.679870440984478, 1.9488185499908899, 0, 0]
Guessing letter: r
Sever response: {'game_id': 'f33e7647b251', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ r _ _ e r _ _ _ _ '}
i 1.42, a 1.4, o 1.2, t 1.18, s 1.04, n 0.93, l 0.77, c 0.62, d 0.59, p 0.58, u 0.51, m 0

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)